In [5]:
%%writefile ../src/BankV3.sol
pragma solidity ^0.6.0;

contract BankV3 {
    address owner;
    uint balance;
    uint256 timeToDeposit;
    
    event PrintLog(string);
    event Sent(address from, address to, uint amount );
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }
//    function() external {
//        emit PrintLog("Fallback called");
//    }
    function forwardTo(address payable _receiver) public payable onlyOwner {
        //require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable onlyAfter {
        timeToDeposit = now + 10 seconds;
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public onlyOwner minBalance {
        balance -= address(this).balance;
        //require(msg.sender == owner);
        msg.sender.transfer(address(this).balance);
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
    modifier onlyAfter {
        require(now >= timeToDeposit);
        _;
    }
    modifier minBalance {
        require(address(this).balance>101 wei);
        _;
    }
}

Overwriting ../src/BankV3.sol


In [6]:
!solc --gas --abi --bin ../src/BankV3.sol


======= ../src/BankV3.sol:BankV3 =======
Gas estimation:
construction:
   26100 + 178600 = 204700
external:
   deposit(uint256):	41947
   forwardTo(address):	infinite
   getBalance():	1027
   withdrawAll():	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550600060018190555061037d806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad8814610076578063853828b6146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610250565b005b6100fd600480360360208110156100e757600080fd5b810190808035906020019092919050505061030f565b005b60008060015447915091509091565b6000809054906101000a90047

In [8]:
%%writefile ../src/BankV3Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abiArray = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550600060018190555061037d806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad8814610076578063853828b6146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610250565b005b6100fd600480360360208110156100e757600080fd5b810190808035906020019092919050505061030f565b005b60008060015447915091509091565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461016757600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156101ad573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146102a957600080fd5b606547116102b657600080fd5b476001600082825403925050819055503373ffffffffffffffffffffffffffffffffffffffff166108fc479081150290604051600060405180830381858888f1935050505015801561030c573d6000803e3d6000fd5b50565b60025442101561031e57600080fd5b600a420160028190555080341461033457600080fd5b806001600082825401925050819055505056fea264697066735822122014fc9b7941c792d1d5826db5f78ba6d268f2ef9a85caf637fd93c7d3f22110be64736f6c63430006040033";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/BankV3Deploy.js


In [9]:
!node ../src/BankV3Deploy.js

Deploying the contract from 0x2EE1Ff8309bA4D85276A9C68f515E308774BF8a1
hash: 0x2b380cf81a9bbec62587e08f283eb1b1493fb0f3a7bc5552093b236fc5151713
---> The contract deployed to: 0x01bBa0e2314082459a112e37Df794ED382Dc826C


In [10]:
%%writefile ../src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, "0x01bBa0e2314082459a112e37Df794ED382Dc826C");


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    bank.methods.getBalance().call().then(console.log);
    
    await bank.methods.deposit(111).send({from: accounts[0], value:111});
    bank.methods.getBalance().call().then(console.log);
    
    await bank.methods.withdrawAll().send({from: accounts[0]});    //greater than 101
    bank.methods.getBalance().call().then(console.log);
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}

doIt()

Writing ../src/BankV3Use.js


In [11]:
!node ../src/BankV3Use.js

Account: 0x2EE1Ff8309bA4D85276A9C68f515E308774BF8a1
Balance before: 99994623200000000000
Result { '0': '0', '1': '0' }
Result { '0': '111', '1': '111' }
Result { '0': '0', '1': '0' }
Balance after: 99992950880000000000
Balance diff: 1672320000000000


## 이벤트

In [1]:
%%writefile ../src/EventTest.sol
pragma solidity ^0.6.0;
contract EventTest {
    event MyLog(string my);
    function myFunction() public {
        emit MyLog("Hello World!");
    }
}

Writing ../src/EventTest.sol


In [2]:
!solc --abi --bin --gas ../src/EventTest.sol


======= ../src/EventTest.sol:EventTest =======
Gas estimation:
construction:
   93 + 42600 = 42693
external:
   myFunction():	1748
Binary:
6080604052348015600f57600080fd5b5060d58061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea2646970667358221220ff7b72290a2cdb2fbee8af39cca06e50ce52c3f07da773ec07a75843a5e810a664736f6c63430006040033
Contract JSON ABI
[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}]


In [5]:
%%writefile ../src/EventTestDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray  = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x" + "6080604052348015600f57600080fd5b5060d58061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea2646970667358221220ff7b72290a2cdb2fbee8af39cca06e50ce52c3f07da773ec07a75843a5e810a664736f6c63430006040033";

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/EventTestDeploy.js


In [6]:
!node ../src/EventTestDeploy.js

Deploying the contract from 0x2f0b846926ee89D9dF4e0F545Fde07ED69b2F3b2
hash: 0x980c0e54decb36d2c5c0f7d621db8a58a49d19d05a8ce65946b3c3f5eab22de7
---> The contract deployed to: 0xD9735d6Eb067f5143303C56e5131Db78a2f08bA0


### 처리결과 보기 reciept

In [10]:
!geth --exec "eth.getTransactionReceipt('0x980c0e54decb36d2c5c0f7d621db8a58a49d19d05a8ce65946b3c3f5eab22de7');" attach http://localhost:8345

{
  blockHash: "0x802b0ae3b89297b0c41c302482924a0de032cf83d293226dccfa2c4589d1435c",
  blockNumber: 1,
  contractAddress: "0xd9735d6eb067f5143303c56e5131db78a2f08ba0",
  cumulativeGasUsed: 99221,
  from: "0x2f0b846926ee89d9df4e0f545fde07ed69b2f3b2",
  gasUsed: 99221,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0x980c0e54decb36d2c5c0f7d621db8a58a49d19d05a8ce65946b3c3f5eab22de7",
  transactionIndex: 0
}


In [13]:
%%writefile ../src/EventTestHttpNoEventFiredUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _test = new web3.eth.Contract(_abiArray, '0xD9735d6Eb067f5143303C56e5131Db78a2f08bA0');
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting ../src/EventTestHttpNoEventFiredUse.js


In [14]:
!node ../src/EventTestHttpNoEventFiredUse.js

Account: 0x2f0b846926ee89D9dF4e0F545Fde07ED69b2F3b2
Balance before: 99999900779000000000
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99999877967000000000
Balance diff: 22812000010240


In [1]:
%%writefile ../src/EventTestWsUse.js
var Web3=require('web3');
var fs=require('fs');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];

async function doIt() {
    var _test = new web3.eth.Contract(_abiArray, '0xD9735d6Eb067f5143303C56e5131Db78a2f08bA0');
    var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            //fs.writeFile("src/EventTestLog.txt", log, "utf-8", function(e) {
            fs.appendFile("src/EventTestLog.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting ../src/EventTestWsUse.js


In [ ]:
!node ../src/EventTestWsUse.js